In [1]:
import transformers
import gandalf_dataset
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import peft
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import wandb
import hyper_params as h

/common/home/projectgrps/CS605/CS605G5/jupyterlab-venv-pytorch-py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!wandb login c6519e6832b84377faa436c31781e3c236fa0504

wandb: Appending key for api.wandb.ai to your netrc file: /common/home/projectgrps/CS605/CS605G5/.netrc


In [3]:
#configure distributed data parallel
is_ddp = int(os.environ.get("WORLD_SIZE", 1)) != 1
ds = gandalf_dataset.TrainDataset()
collator = transformers.DataCollatorForSeq2Seq(ds.tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True)
device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)} if is_ddp else None

#configure 4bit quantisation
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

#create 4bit model
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-hf", quantization_config=bnb_config, device_map=device_map)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.61s/it]
/common/home/projectgrps/CS605/CS605G5/jupyterlab-venv-pytorch-py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/common/home/projectgrps/CS605/CS605G5/jupyterlab-venv-pytorch-py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `

In [4]:
#preprocessing for training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
#configure lora
config = LoraConfig(
    r=h.LORA_R, 
    lora_alpha=h.LORA_ALPHA, 
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=h.LORA_DROPOUT, 
    bias="none", 
    task_type="CAUSAL_LM"
)

#add lora adaptors to model
model = get_peft_model(model, config)

In [6]:
#define training parameters
trainer = transformers.Trainer(
  model=model,
  train_dataset=ds,
  data_collator=collator,
  args=transformers.TrainingArguments(
    per_device_train_batch_size= h.TRAIN_BATCH_SIZE,
    num_train_epochs=h.TRAIN_NUM_EPOCHS,
    learning_rate= h.TRAIN_LEARNING_RATE,
    fp16=True,
    logging_steps= h.TRAIN_LOGGING_STEPS,
    optim="paged_adamw_8bit",
    eval_strategy="no",
    save_strategy="steps",
    eval_steps=None,
    save_steps= h.TRAIN_SAVE_STEPS,
    output_dir="./genius",
    save_total_limit= h.TRAIN_SAVE_TOTAL_LIMIT,
    report_to="wandb",
    ddp_find_unused_parameters=False if is_ddp else None,
  ),
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [7]:
model.config.use_cache = False
wandb.finish()
trainer.train()
model.save_pretrained("./weights_tests")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: zitian-wang-2023 (smu-mitb-nlp-2024-g5). Use `wandb login --relogin` to force relogin


/common/home/projectgrps/CS605/CS605G5/jupyterlab-venv-pytorch-py311/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.15 GiB. GPU 